# Disturbance & Event Guides: RainfallAgent

This notebook provides a guide to the `RainfallAgent`. This agent is used to simulate a rainfall event by providing a time series of precipitation data to the simulation.

## 1. How It Works

The `RainfallAgent` is a simple disturbance agent. It is initialized with a `rainfall_pattern`, which is a list of rainfall depth values (e.g., in mm). At each time step of the simulation, the agent outputs the next value from this list. 

This agent is designed to be connected to a hydrology model (like `SCSRunoffModel` or `XinanjiangModel`) to simulate the process of runoff generation from rainfall.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.disturbances.agents import RainfallAgent
from water_system_sdk.src.chs_sdk.modules.modeling.hydrology.runoff_models import SCSRunoffModel

## 2. Code Example: A Complete Rainfall-Runoff Simulation

We will create a full simulation that connects a `RainfallAgent` to an `SCSRunoffModel`. This will show the end-to-end process of converting a storm event into a runoff hydrograph.

In [ ]:
# 1. Define the rainfall pattern (a simple storm hyetograph)
rainfall_pattern = [0, 0, 5, 10, 15, 25, 18, 12, 8, 4, 2, 1, 0, 0]

# 2. Initialize the models
rainfall_agent = RainfallAgent(rainfall_pattern=rainfall_pattern)
scs_model = SCSRunoffModel()

# 3. Simulation Setup
dt = 1.0 # Assume 1-hour time step, matching the pattern
n_steps = len(rainfall_pattern)
scs_params = {"CN": 85} # High runoff potential

history = {
    "time": list(range(n_steps)),
    "rainfall": [],
    "runoff": []
}

# 4. Simulation Loop
for t in range(n_steps):
    # Get the current rainfall from the agent
    rainfall_agent.step(t=t, dt=dt)
    current_rainfall = rainfall_agent.output
    
    # Use the rainfall as input to the SCS model
    current_runoff = scs_model.calculate_runoff(
        rainfall=current_rainfall, 
        sub_basin_params=scs_params, 
        dt=dt
    )
    
    # Record results
    history["rainfall"].append(current_rainfall)
    history["runoff"].append(current_runoff)

print("Simulation complete.")

## 3. Visualization

Plotting the rainfall hyetograph (bar chart) and the resulting runoff hydrograph (line chart) shows the direct relationship.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Rainfall as a bar chart (hyetograph)
color = 'tab:blue'
ax1.set_xlabel('Time (hours)')
ax1.set_ylabel('Rainfall (mm)', color=color)
ax1.bar(history['time'], history['rainfall'], color=color, alpha=0.6, label='Rainfall')
ax1.tick_params(axis='y', labelcolor=color)

# Plot Runoff on a second y-axis
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Direct Runoff (mm)', color=color)
ax2.plot(history['time'], history['runoff'], color=color, label='Runoff')
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(bottom=0) # Ensure runoff axis starts at 0

plt.title('Rainfall-Runoff Simulation using RainfallAgent and SCSRunoffModel')
fig.tight_layout()
# Manually create legends because of the twin axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)
plt.show()

The plot shows that runoff (the red line) is only generated during the hours with significant rainfall. The shape of the runoff hydrograph directly follows the shape of the rainfall hyetograph, with magnitudes determined by the SCS model's logic. This example shows how disturbance agents can be chained together with physics models to create complete, meaningful simulations.